# Can Congress and Twitter data be linked?

>How to use Congress API: https://github.com/eyeseast/propublica-congress/blob/master/docs/api.rst

In [129]:
import tweepy
import pandas as pd

## Set up Congress API: Senate

In [130]:
from congress import Congress

congress = Congress('qa1qfTzGinGGLcNCbinp2OTlI2SvzBe4KFgxYobm')

# get member by bioguide ID
senate = congress.members.filter('senate')

In [131]:
#access members in dict
sen_members = senate[0]['members']

#save in df
df_senate = pd.DataFrame(sen_members)
df_senate.columns #twitter_account gives twitter name

Index(['api_uri', 'contact_form', 'crp_id', 'cspan_id', 'date_of_birth',
       'dw_nominate', 'facebook_account', 'fax', 'fec_candidate_id',
       'first_name', 'gender', 'google_entity_id', 'govtrack_id', 'icpsr_id',
       'id', 'ideal_point', 'in_office', 'last_name', 'leadership_role',
       'lis_id', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'senate_class', 'seniority', 'short_title', 'state', 'state_rank',
       'suffix', 'title', 'total_present', 'total_votes', 'twitter_account',
       'url', 'votes_with_party_pct', 'votesmart_id', 'youtube_account'],
      dtype='object')

In [132]:
total = df_senate.shape[0]

In [263]:
print ("Share of Senators without Twitter account:", (df_senate.twitter_account.isnull().sum()/total)*100,'\n'
       "Share of Representatives without Twitter account:", (df_house.twitter_account.isnull().sum()/total)*100,'\n'
      )

Share of Senators without Twitter account: 3.8461538461538463 
Share of Representatives without Twitter account: 9.615384615384617 



>Good News: only 3% do not have a twitter account (Senate) and 10% (House)

>*What are other stats that are valuable in the Congress data?*

# Set up House of Reps

In [298]:
df_house = pd.DataFrame(congress.members.filter('house')[0]['members'])
df_house.shape

(448, 43)

In [342]:
import time
from datetime import datetime

In [385]:
#take id column and look for data with members.get(id)
def recent_vote(rep):
    try:
        #get the date
        dt = datetime.strptime(congress.members.get(df_house.id.iloc[0])['most_recent_vote'], '%Y-%m-%d')
        return dt.year, dt.month
    except:
        time.sleep(10)
        print("Error")

In [386]:
recent_vote(df_house.id.iloc[0])

(2018, 4)

In [391]:
congress.members.get('R000570')

{'crp_id': 'N00004357',
 'cspan_id': '57970',
 'current_party': 'R',
 'date_of_birth': '1970-01-29',
 'facebook_account': 'speakerryan',
 'first_name': 'Paul',
 'gender': 'M',
 'google_entity_id': '/m/024v2j',
 'govtrack_id': '400351',
 'icpsr_id': '29939',
 'in_office': True,
 'last_name': 'Ryan',
 'member_id': 'R000570',
 'middle_name': 'D.',
 'most_recent_vote': '2018-04-27',
 'roles': [{'at_large': False,
   'bills_cosponsored': 2,
   'bills_sponsored': 2,
   'chamber': 'House',
   'committees': [],
   'congress': '115',
   'contact_form': None,
   'district': '1',
   'end_date': '2019-01-03',
   'fax': '202-225-3393',
   'fec_candidate_id': 'H8WI01024',
   'leadership_role': 'Speaker of the House',
   'missed_votes_pct': 97.27,
   'ocd_id': 'ocd-division/country:us/state:wi/cd:1',
   'office': '1233 Longworth House Office Building',
   'party': 'R',
   'phone': '202-225-3031',
   'seniority': '20',
   'short_title': 'Rep.',
   'start_date': '2017-01-03',
   'state': 'WI',
   'subc

In [388]:
df_house['recent_vote'] = df_house.id.apply(recent_vote)

Error


,api_uri,at_large,contact_form,crp_id,cspan_id,date_of_birth,district,dw_nominate,facebook_account,fax,...,suffix,title,total_present,total_votes,twitter_account,url,votes_with_party_pct,votesmart_id,youtube_account,recent_vote
0,https://api.propublica.org/congress/v1/members...,False,None,N00036633,76236,1954-09-16,5,0.497,CongressmanRalphAbraham,202-225-5639,...,None,Representative,0.0,878.0,RepAbraham,https://abraham.house.gov,97.01,155414,None,"(2018, 4)"
1,https://api.propublica.org/congress/v1/members...,False,None,N00035451,76386,1946-05-27,12,-0.465,CongresswomanAdams,202-225-1512,...,None,Representative,0.0,878.0,RepAdams,https://adams.house.gov,95.69,5935,None,"(2018, 4)"
2,https://api.propublica.org/congress/v1/members...,False,None,N00003028,45516,1965-07-22,4,0.360,RobertAderholt,202-225-5587,...,None,Representative,0.0,878.0,Robert_Aderholt,https://aderholt.house.gov,96.58,441,RobertAderholt,"(2018, 4)"
3,https://api.propublica.org/congress/v1/members...,False,None,N00033997,79994,1979-06-19,31,-0.284,reppeteaguilar,202-226-6962,...,None,Representative,0.0,878.0,reppeteaguilar,https://aguilar.house.gov,91.94,70114,None,"(2018, 4)"
4,https://api.propublica.org/congress/v1/members...,False,None,N00033720,62545,1951-11-07,12,0.626,CongressmanRickAllen,202-225-3377,...,None,Representative,0.0,878.0,reprickallen,https://allen.house.gov,97.71,136062,None,"(2018, 4)"
5,https://api.propublica.org/congress/v1/members...,False,None,N00031938,1033767,1980-04-18,3,0.658,repjustinamash,202-225-5144,...,None,Representative,0.0,878.0,None,https://amash.house.gov,71.72,105566,repjustinamash,"(2018, 4)"
6,https://api.propublica.org/congress/v1/members...,False,None,N00031177,62817,1958-06-12,2,0.378,MarkAmodeiNV2,202-225-5679,...,None,Representative,1.0,878.0,MarkAmodeiNV2,https://amodei.house.gov,95.88,12537,markamodeinv2,"(2018, 4)"
7,https://api.propublica.org/congress/v1/members...,False,None,N00038285,1016482,1972-03-09,19,0.555,JodeyArrington,None,...,None,Representative,0.0,878.0,RepArrington,https://arrington.house.gov,98.39,155685,None,"(2018, 4)"
8,https://api.propublica.org/congress/v1/members...,False,None,N00005736,44883,1948-03-23,36,0.717,RepBrianBabin,202-226-0396,...,None,Representative,0.0,878.0,RepBrianBabin,https://babin.house.gov,95.36,360,None,"(2018, 4)"
9,https://api.propublica.org/congress/v1/members...,False,None,N00037049,103442,1963-08-16,2,0.442,None,None,...,None,Representative,0.0,878.0,RepDonBacon,https://bacon.house.gov,96.01,166299,None,"(2018, 4)"


In [389]:
df_house

,api_uri,at_large,contact_form,crp_id,cspan_id,date_of_birth,district,dw_nominate,facebook_account,fax,...,suffix,title,total_present,total_votes,twitter_account,url,votes_with_party_pct,votesmart_id,youtube_account,recent_vote
0,https://api.propublica.org/congress/v1/members...,False,None,N00036633,76236,1954-09-16,5,0.497,CongressmanRalphAbraham,202-225-5639,...,None,Representative,0.0,878.0,RepAbraham,https://abraham.house.gov,97.01,155414,None,"(2018, 4)"
1,https://api.propublica.org/congress/v1/members...,False,None,N00035451,76386,1946-05-27,12,-0.465,CongresswomanAdams,202-225-1512,...,None,Representative,0.0,878.0,RepAdams,https://adams.house.gov,95.69,5935,None,"(2018, 4)"
2,https://api.propublica.org/congress/v1/members...,False,None,N00003028,45516,1965-07-22,4,0.360,RobertAderholt,202-225-5587,...,None,Representative,0.0,878.0,Robert_Aderholt,https://aderholt.house.gov,96.58,441,RobertAderholt,"(2018, 4)"
3,https://api.propublica.org/congress/v1/members...,False,None,N00033997,79994,1979-06-19,31,-0.284,reppeteaguilar,202-226-6962,...,None,Representative,0.0,878.0,reppeteaguilar,https://aguilar.house.gov,91.94,70114,None,"(2018, 4)"
4,https://api.propublica.org/congress/v1/members...,False,None,N00033720,62545,1951-11-07,12,0.626,CongressmanRickAllen,202-225-3377,...,None,Representative,0.0,878.0,reprickallen,https://allen.house.gov,97.71,136062,None,"(2018, 4)"
5,https://api.propublica.org/congress/v1/members...,False,None,N00031938,1033767,1980-04-18,3,0.658,repjustinamash,202-225-5144,...,None,Representative,0.0,878.0,None,https://amash.house.gov,71.72,105566,repjustinamash,"(2018, 4)"
6,https://api.propublica.org/congress/v1/members...,False,None,N00031177,62817,1958-06-12,2,0.378,MarkAmodeiNV2,202-225-5679,...,None,Representative,1.0,878.0,MarkAmodeiNV2,https://amodei.house.gov,95.88,12537,markamodeinv2,"(2018, 4)"
7,https://api.propublica.org/congress/v1/members...,False,None,N00038285,1016482,1972-03-09,19,0.555,JodeyArrington,None,...,None,Representative,0.0,878.0,RepArrington,https://arrington.house.gov,98.39,155685,None,"(2018, 4)"
8,https://api.propublica.org/congress/v1/members...,False,None,N00005736,44883,1948-03-23,36,0.717,RepBrianBabin,202-226-0396,...,None,Representative,0.0,878.0,RepBrianBabin,https://babin.house.gov,95.36,360,None,"(2018, 4)"
9,https://api.propublica.org/congress/v1/members...,False,None,N00037049,103442,1963-08-16,2,0.442,None,None,...,None,Representative,0.0,878.0,RepDonBacon,https://bacon.house.gov,96.01,166299,None,"(2018, 4)"


## Set up Tweepy

In [176]:
from tweepy import OAuthHandler
import numpy as np
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

In [177]:
consumer_key= 'AKEpYmXTTBDu6lePXsBfOz3zy'
consumer_secret = 'h6Kz7HPfmGVCVqsKeck6kSwOsIxnIHeMpEy6NOUkfSYHjF8NMJ'
access_token="991768520546488321-OvBfDZg6YEZpo3oWnabsOj5rhRBCNvY"
access_token_secret="wLJUj2hDB0wE4A4x2TgNnesEqFmqHbyRlNvMorWjcoMfw"
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

In [178]:
#save account names in list an skip 'None'
account_list = [i for i in df_senate['twitter_account'] if not i == None] 
account_list[:3]

['SenAlexander', 'SenatorBaldwin', 'SenJohnBarrasso']

In [184]:
#average tweets per day
def tweets_dagen(name):
    try:
        if not name == None: 
            user = auth_api.get_user(name)
            tweets = user.statuses_count
            account_created_date = user.created_at
            delta = datetime.utcnow() - account_created_date 
            return (tweets/delta.days)
        else:
            return print ("Ikke noget navn")
    except:
        return print ("User not found")

In [245]:
#calculate average tweets per day
df_senate['tweets_dagen'] = df_senate.twitter_account.apply(tweets_dagen) 
df_house['tweets_dagen'] = df_house.twitter_account.apply(tweets_dagen)

Ikke noget navn
Ikke noget navn
En fejl
Ikke noget navn
Ikke noget navn
En fejl
En fejl
Ikke noget navn
Ikke noget navn
Ikke noget navn
Ikke noget navn
En fejl
En fejl
Ikke noget navn
Ikke noget navn
Ikke noget navn
En fejl
En fejl
En fejl
Ikke noget navn
En fejl
Ikke noget navn
Ikke noget navn


In [250]:
print ("Share of inactive Twitter accounts (Senate):", (df_senate.tweets_dagen.isnull().sum()/df_senate.shape[0])*100)
print ("Share of inactive Twitter accounts (House):", (df_house.tweets_dagen.isnull().sum()/df_house.shape[0])*100)

Share of inactive Twitter accounts (Senate): 6.730769230769231
Share of inactive Twitter accounts (House): 3.571428571428571


In [262]:
#The 10 most active users
    #Senate
print ("---Senate")
print ("5 most active users:\n", df_senate.tweets_dagen.sort_values(ascending=False).iloc[:5])
print ("Average Tweets per day among twitter users: ", df_senate.tweets_dagen.mean(), '\n')

    #House
print ("---House of representatives")
print ("5 most active users: \n",df_house.tweets_dagen.sort_values(ascending=False).iloc[:5])
print ("Average Tweets per day among twitter users: ", df_house.tweets_dagen.mean())

---Senate
5 most active users:
 6     17.362152
88     7.500000
20     7.226117
1      5.959773
21     5.789157
Name: tweets_dagen, dtype: float64
Average Tweets per day among twitter users:  2.5608118681483085 

---House of representatives
5 most active users: 
 194    20.460905
20     10.852432
125    10.070539
213     8.309917
329     8.007843
Name: tweets_dagen, dtype: float64
Average Tweets per day among twitter users:  1.8391823073634934


### Plot some descriptives  

In [214]:
import seaborn as sns

#### Infos about followers

In [105]:
def get_follower_ids(target):
    return auth_api.followers_ids(target)

len(get_follower_ids(account_list[0])) #somehow stopping at 5000!

>fetches only the most recent 5000 followers

 > try sth else

In [240]:
auth_api.followers(account_list[0])

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]